### Logging in to hf and wandb

In [1]:
from huggingface_hub import login
import wandb

with open("../api/hf.txt", "r") as f:
    hf_token = f.read().strip()

with open("../api/wandb.txt", "r") as f:
    wandb_token = f.read().strip()
    
login(token=hf_token)
wandb.login(key=wandb_token)

c:\Users\hamid\OneDrive\Documents\ai-surveyor\env\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: C:\Users\hamid\_netrc
wandb: Currently logged in as: awsed-aq (awsed-aq-lut-university) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


True

In [3]:
run = wandb.init(
    project='DeepSeek-R1-Distill-Llama-8B-ft for Surveying', 
    job_type="training", 
    anonymous="allow"
)

wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


### Importing necessary libraries
Also declaring the local directory

In [1]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM

local_directory = "../local_model"
tokenizer_path = local_directory + "/tokenizer"
model_path = local_directory + "/model"

c:\Users\hamid\OneDrive\Documents\ai-surveyor\env\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### Loading the model from huggingface (For the first time only)

In [ ]:
model_name = "deepseek-ai/DeepSeek-R1-Distill-Qwen-1.5B"

tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name).to("cuda")

c:\Users\hamid\OneDrive\Documents\ai-surveyor\env\Lib\site-packages\huggingface_hub\file_download.py:140: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\hamid\.cache\huggingface\hub\models--deepseek-ai--DeepSeek-R1-Distill-Qwen-1.5B. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


#### Saving the model in a local directory

In [15]:
tokenizer.save_pretrained(tokenizer_path)
model.save_pretrained(model_path)

#### Checking if CUDA is available

In [16]:
torch.cuda.is_available()

True

In [5]:
model = AutoModelForCausalLM.from_pretrained(
    'deepseek-ai/DeepSeek-R1-Distill-Qwen-1.5B',
    device_map='auto',
    load_in_4bit=True,
    bnb_4bit_quant_type='nf4',
    bnb_4bit_compute_dtype=torch.float16
)
model.save_pretrained(local_directory + '/deepseek-1.5b-4bit')

The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.


In [14]:
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
import torch

# Configure 4-bit loading with CPU offloading
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_use_double_quant=True,
    llm_int8_enable_fp32_cpu_offload=True  # Critical for Windows
)
device_map = {
    "transformer.wte": 0,
    "transformer.h.0": 0,
    "transformer.h.1": 0,
    "transformer.h.2": 0,
    "transformer.h.3": 0,  # First 4 layers on GPU
    "transformer.ln_f": "cpu",  # Later layers on CPU
    "lm_head": "cpu"
}

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline

model = AutoModelForCausalLM.from_pretrained(
    local_directory + '/deepseek-1.5b-4bit',
    device_map=device_map,
    quantization_config=bnb_config,
    offload_folder= local_directory + "/offload",  # Required for Windows
    torch_dtype=torch.float16
)
tokenizer = AutoTokenizer.from_pretrained('deepseek-ai/DeepSeek-R1-Distill-Qwen-1.5B')

# Create optimized pipeline
pipe = pipeline(
    'text-generation',
    model=model,
    tokenizer=tokenizer,
    max_new_tokens=256,
    do_sample=True,
    temperature=0.7,
    pad_token_id=tokenizer.eos_token_id,
    model_kwargs={
        'use_cache': True,
        'attn_implementation': 'sdpa'  # Flash Attention alternative
    }
)


In [ ]:

# Run inference
response = pipe("Explain quantum computing in simple terms:")
print(response)

### Loading the model from the local directory

In [17]:
tokenizer = AutoTokenizer.from_pretrained(tokenizer_path)
model = AutoModelForCausalLM.from_pretrained(model_path).to(torch.device("cuda"))

Loading checkpoint shards: 100%|██████████| 2/2 [00:00<00:00,  6.85it/s]


### Testing the Model 

In [24]:
from transformers import pipeline

# Initialize the text generation pipeline
generator = pipeline('text-generation', model=model, tokenizer=tokenizer)


Device set to use cuda:0


What type of lubricant is recommended for restoring old camera mechanisms? I'm a bit new to this, so I need to figure it out step by step. Maybe I can start by identifying the type of mechanism in the camera. Then, perhaps look into the materials used in these mechanisms. After that, I should find out the specific conditions the mechanism was in when it was taken out of the camera, like temperature, pressure, etc. Then, figure out what kind of lubricant is recommended for that scenario. Maybe I can think of some examples. For example, if the mechanism was in a high-temperature environment, maybe a silicone-based lubricant is good. Or if it was in a dry environment, maybe a silicone-based or oil-based. But I'm not sure. Maybe I should look up some examples to get a better idea. Also, maybe I should think about the specific parts of the mechanism that are prone to wear and tear, like springs, levers, gears, etc. Then, for each part, I can figure out the type of lubricant that's best. For

In [25]:

# Generate text
prompt = "Hello, how are you?"
generated_text = generator(prompt, max_length=300, num_return_sequences=1)

# Print the generated text
print(generated_text)

[{'generated_text': 'Hello, how are you? I have this question about a certain function. Let me try to think through it step by step.\n\nAlright, the question is: Let f(x) = x^2 + 2x + 3. Let g(x) be a function such that g(x) = f(x + a) + f(x - a) for some constant a. What is the value of g(0)?\n\nOkay, so I need to figure out g(0). Let me first write down what g(x) is. It says g(x) = f(x + a) + f(x - a). So, f is given as x squared plus 2x plus 3. So, I need to compute f(x + a) and f(x - a), add them together, and then evaluate that at x = 0.\n\nLet me write down f(x + a). Since f(x) is x^2 + 2x + 3, replacing x with (x + a) gives:\n\nf(x + a) = (x + a)^2 + 2(x + a) + 3.\n\nSimilarly, f(x - a) = (x - a)^2 + 2(x - a) + 3.\n\nSo, g(x) = f(x + a) + f(x - a). Let me compute each term separately.\n\nFirst, expand f(x + a):\n\n(x + a)^2 = x^2 + '}]


: 